In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [71]:
index = '^CNXIT'
index_data = yf.download(auto_index,period='max',interval='1mo')

[*********************100%***********************]  1 of 1 completed


In [72]:
df = pd.DataFrame()
df['Close']=index_data['Close']
df.reset_index(inplace=True)
df['Month'] = pd.DatetimeIndex(df['Date']).month
df['Month'] = df['Month'].apply(lambda x: datetime(2022, x, 1).strftime('%B'))
df['Year'] = pd.DatetimeIndex(df['Date']).year
df['Direction'] = (df['Close'] > df['Close'].shift(1)).map({True: 1, False: 0})
df.head(20)

,Date,Close,Month,Year,Direction
0,2011-08-01,3469.300049,August,2011,0
1,2011-09-01,3549.620117,September,2011,1
2,2011-10-01,3944.050049,October,2011,1
3,2011-11-01,3527.800049,November,2011,0
4,2011-12-01,3390.550049,December,2011,0
5,2012-01-01,3836.500000,January,2012,1
6,2012-02-01,4159.399902,February,2012,1
7,2012-03-01,4207.000000,March,2012,1
8,2012-04-01,4433.149902,April,2012,1
9,2012-05-01,3739.750000,May,2012,0


In [73]:
mo_order = df.iloc[:12,2].values

In [74]:
mo_order

array(['August', 'September', 'October', 'November', 'December',
       'January', 'February', 'March', 'April', 'May', 'June', 'July'],
      dtype=object)

In [75]:
del(df['Date'])
del(df['Close'])
del(df['Year'])
df.head()

,Month,Direction
0,August,0
1,September,1
2,October,1
3,November,0
4,December,0


In [76]:
X = df.iloc[:,:-1].values
Y= df.iloc[:,-1].values

In [77]:
Y = Y.reshape(-1,1)

In [78]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
#apply onehot encoding to column 1 to transform categorical data into a numeric value
encoder = OneHotEncoder(sparse=False, drop='first')
ct = ColumnTransformer(transformers=[('encoder',encoder,[0])],remainder='passthrough')
X=np.array(ct.fit_transform(X))

/home/glen/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [79]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 0)

In [80]:
# Training the Logistic Regression model on the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

/home/glen/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=0)

In [81]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 5  9]
 [ 1 21]]


0.7222222222222222

In [82]:
pred = classifier.predict(ct.fit_transform(X[:12,:]))
print(np.concatenate((mo_order.reshape(len(mo_order),1), pred.reshape(len(pred),1)),1))

[['August' 1]
 ['September' 1]
 ['October' 1]
 ['November' 1]
 ['December' 1]
 ['January' 1]
 ['February' 0]
 ['March' 0]
 ['April' 1]
 ['May' 1]
 ['June' 1]
 ['July' 1]]


/home/glen/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
